In [1]:
# Cell 1 — Setup & Load
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import lightgbm as lgb
import joblib, warnings

warnings.filterwarnings("ignore", category=FutureWarning)

# Where to look for the CSV
CANDIDATE_PATHS = [
    "./data/raw/cargurus.csv",
    "../data/raw/cargurus.csv",
    "cargurus.csv",
]

def load_csv():
    for p in CANDIDATE_PATHS:
        p = Path(p)
        if p.exists():
            print(f"[data] {p.resolve()}")
            return pd.read_csv(p, low_memory=False)
    raise FileNotFoundError(f"Could not find cargurus.csv in: {CANDIDATE_PATHS}")

df = load_csv()
print("Raw shape:", df.shape)
df.head(3)


[data] C:\Users\junok\car-price-predictor\data\raw\cargurus.csv
Raw shape: (3000040, 66)


,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,ZACNJABB5KPJ92081,35.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,NaN,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,SALCJ2FX1LH858117,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,JF1VA2M67G9829723,35.4 in,NaN,NaN,NaN,Sedan,NaN,Guaynabo,17.0,NaN,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016


In [2]:
# Cell 2 — Keep only needed columns & fix types
required = ["year", "mileage", "price", "model_name"]
optional = ["engine_displacement", "trim_name", "has_accidents", "salvage", "theft_title"]

missing = [c for c in required if c not in df.columns]
if missing:
    raise KeyError(f"Missing required columns: {missing}")

keep_cols = required + [c for c in optional if c in df.columns]
df = df[keep_cols].copy()

# Numeric normalize
for c in ["year", "mileage", "price", "engine_displacement"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Boolean flags (if present) -> 0/1
for b in ["has_accidents", "salvage", "theft_title"]:
    if b in df.columns:
        df[b] = df[b].replace({True:1, False:0, "True":1, "False":0})
        df[b] = pd.to_numeric(df[b], errors="coerce").fillna(0).astype(int)

# Strings
df["model_name"] = df["model_name"].fillna("Unknown").astype(str)
df["trim_name"]  = df.get("trim_name", "Unknown").fillna("Unknown").astype(str)

# Engine displacement — create if missing, then fill median later
if "engine_displacement" not in df.columns:
    df["engine_displacement"] = np.nan

print(df.dtypes)
df.head(3)


year                     int64
mileage                float64
price                  float64
model_name              object
engine_displacement    float64
trim_name               object
has_accidents            int32
salvage                  int32
theft_title              int32
dtype: object


,year,mileage,price,model_name,engine_displacement,trim_name,has_accidents,salvage,theft_title
0,2019,7.0,23141.0,Renegade,1300.0,Latitude FWD,0,0,0
1,2020,8.0,46500.0,Discovery Sport,2000.0,S AWD,0,0,0
2,2016,NaN,46995.0,WRX STI,2500.0,Base,0,0,0


In [3]:
# Cell 3 — Cleaning: fill NAs, filter years, remove outliers
YEAR_RANGE = (1990, 2021)
ABS_LOWER_Q, ABS_UPPER_Q = 0.005, 0.995
REL_LOWER_Q, REL_UPPER_Q = 0.05, 0.95

# Fill numerics with medians (temporary)
for c in ["year", "mileage", "price", "engine_displacement"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")
        med = df[c].median()
        df[c] = df[c].fillna(med)

# Year filter
ymin, ymax = YEAR_RANGE
before = len(df)
df = df[(df["year"] >= ymin) & (df["year"] <= ymax)].copy()
print(f"[year-filter] {before} -> {len(df)} rows")

# Absolute outliers on price and mileage
p_lo = df["price"].quantile(ABS_LOWER_Q)
p_hi = df["price"].quantile(ABS_UPPER_Q)
m_lo = df["mileage"].quantile(ABS_LOWER_Q)
m_hi = df["mileage"].quantile(ABS_UPPER_Q)
m_lo = max(m_lo, 100)  # physical sanity

before = len(df)
df = df[
    df["price"].between(p_lo, p_hi) &
    df["mileage"].between(m_lo, m_hi)
].copy()
print(f"[abs-outliers] {before} -> {len(df)} rows | price=[{p_lo:,.0f},{p_hi:,.0f}] mileage=[{m_lo:,.0f},{m_hi:,.0f}]")

# Relational outliers by mileage bins
bins = [0, 20_000, 40_000, 60_000, 80_000, 100_000, 120_000, 150_000, 200_000, np.inf]
labels = ['0-20k','20-40k','40-60k','60-80k','80-100k','100-120k','120-150k','150-200k','200k+']

df["mileage_bin"] = pd.cut(df["mileage"], bins=bins, labels=labels, right=False)
df["price_lb"] = df.groupby("mileage_bin")["price"].transform(lambda x: x.quantile(REL_LOWER_Q))
df["price_ub"] = df.groupby("mileage_bin")["price"].transform(lambda x: x.quantile(REL_UPPER_Q))

before = len(df)
df = df[df["price"].between(df["price_lb"], df["price_ub"])].copy()
df.drop(columns=["mileage_bin","price_lb","price_ub"], inplace=True, errors="ignore")
print(f"[rel-outliers] -> {len(df)} rows")

# Final median fill for engine_displacement after trims
df["engine_displacement"] = pd.to_numeric(df["engine_displacement"], errors="coerce")
df["engine_displacement"] = df["engine_displacement"].fillna(df["engine_displacement"].median())

print("Cleaned shape:", df.shape)
df.head(3)


[year-filter] 3000040 -> 2994755 rows
[abs-outliers] 2994755 -> 1694824 rows | price=[3,500,102,149] mileage=[100,209,225]
[rel-outliers] -> 1527189 rows
Cleaned shape: (1527189, 9)


,year,mileage,price,model_name,engine_displacement,trim_name,has_accidents,salvage,theft_title
2,2016,8267.0,46995.0,WRX STI,2500.0,Base,0,0,0
12,2016,6903.0,58995.0,3 Series,3000.0,340i xDrive Sedan AWD,0,0,0
14,2019,204.0,23695.0,MAZDA3,2500.0,Sedan FWD,0,0,0


In [9]:
# Cell 4 — Derived features (ADD damage_score)
CURRENT_YEAR = 2025

df["car_age"] = (CURRENT_YEAR - df["year"]).clip(lower=0)
df["miles_per_year"] = df["mileage"] / df["car_age"].replace(0, np.nan)
df["miles_per_year"] = df["miles_per_year"].fillna(0)
df["log_mileage"] = np.log1p(df["mileage"])

# --- NEW: combine binary flags into a single risk score ---
for b in ["has_accidents", "salvage", "theft_title"]:
    if b in df.columns:
        df[b] = pd.to_numeric(df[b], errors="coerce").fillna(0).astype(int)
df["damage_score"] = 0
for b in ["has_accidents", "salvage", "theft_title"]:
    if b in df.columns:
        df["damage_score"] += df[b]
# clamp just in case (0~3)
df["damage_score"] = df["damage_score"].clip(0, 3)


In [ ]:
# Cell 5 — Target encoding (leakage-safe) + split + damage_score
# --- 0) Ensure binary flags are clean ints ---
for b in ["has_accidents", "salvage", "theft_title"]:
    if b in df.columns:
        df[b] = pd.to_numeric(df[b], errors="coerce").fillna(0).astype(int)

# --- 1) Add damage_score (sum of available flags; 0~3) ---
flags = [c for c in ["has_accidents", "salvage", "theft_title"] if c in df.columns]
if flags:
    df["damage_score"] = df[flags].sum(axis=1).clip(0, 3)
else:
    df["damage_score"] = 0

# --- 2) Build TE key and collapse rare categories ---
df["model_trim"] = df["model_name"].astype(str) + "_" + df["trim_name"].astype(str)
counts = df["model_trim"].value_counts()
df["model_trim"] = df["model_trim"].replace(counts[counts < 20].index, "_OTHER_")

# --- 3) Prepare X / y (NO price_per_mile) ---
y_all = np.log1p(df["price"])
X_all = df.drop(columns=["price", "major_options"], errors="ignore").copy()

# --- 4) Split (train-only stats for TE) ---
X_tr, X_va, y_tr, y_va = train_test_split(X_all, y_all, test_size=0.2, random_state=42)

# --- 5) Target encoding on TRAIN ONLY ---
enc_map = y_tr.groupby(X_tr["model_trim"]).mean()
X_tr["model_trim_encoded"] = X_tr["model_trim"].map(enc_map)
X_va["model_trim_encoded"] = X_va["model_trim"].map(enc_map).fillna(y_tr.mean())

# --- 6) Drop raw string cols / any leakage-y leftovers ---
drop_strs = ["model_name", "trim_name", "model_trim", "price_per_mile"]  # ensure no price-based feature
X_tr = X_tr.drop(columns=[c for c in drop_strs if c in X_tr.columns], errors="ignore")
X_va = X_va.drop(columns=[c for c in drop_strs if c in X_va.columns], errors="ignore")

# --- 7) Safety: numeric-only features ---
def _is_num(s): 
    return str(s) in ("int64","int32","float64","float32","bool")
bad_tr = [c for c in X_tr.columns if not _is_num(X_tr[c].dtype)]
bad_va = [c for c in X_va.columns if not _is_num(X_va[c].dtype)]
if bad_tr or bad_va:
    raise TypeError(f"Non-numeric features detected. Train: {bad_tr}, Valid: {bad_va}")

# --- 8) Drop low-importance / redundant features ---
drop_low_importance = ["damage_score", "salvage", "theft_title", 'year']
X_tr = X_tr.drop(columns=[c for c in drop_low_importance if c in X_tr.columns], errors="ignore")
X_va = X_va.drop(columns=[c for c in drop_low_importance if c in X_va.columns], errors="ignore")

print("Train columns after drop:", list(X_tr.columns))
print("Train shape:", X_tr.shape, "Valid shape:", X_va.shape)


Train columns after drop: ['year', 'mileage', 'engine_displacement', 'has_accidents', 'car_age', 'miles_per_year', 'log_mileage', 'model_trim_encoded']
Train shape: (1221751, 8) Valid shape: (305438, 8)


In [22]:
# Cell 6 — Train & evaluate (hardened: best_iteration, extra sanity, feature importance)
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
import lightgbm as lgb

# --- Sanity checks on target scaling ---
y_tr_mean_log = float(y_tr.mean())
y_va_mean_log = float(y_va.mean())
y_tr_mean = float(np.expm1(y_tr).mean())
y_va_mean = float(np.expm1(y_va).mean())

print(f"[sanity] y_tr mean (log): {y_tr_mean_log:.4f}  |  expm1 -> ${y_tr_mean:,.0f}")
print(f"[sanity] y_va mean (log): {y_va_mean_log:.4f}  |  expm1 -> ${y_va_mean:,.0f}")

# --- Quick check that TE didn't collapse to constant ---
if "model_trim_encoded" in X_tr.columns:
    te_std = float(X_tr["model_trim_encoded"].std())
    te_na_rate = float(X_tr["model_trim_encoded"].isna().mean() * 100)
    print(f"[sanity] model_trim_encoded std: {te_std:.6f}  |  NaN% in train: {te_na_rate:.2f}%")
    if te_std == 0.0:
        print("⚠️  WARNING: model_trim_encoded has zero variance in TRAIN (likely all global mean).")

# --- LightGBM training ---
LGB_PARAMS = dict(
    objective="regression",
    n_estimators=5000,
    learning_rate=0.03,
    num_leaves=63,
    max_depth=-1,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=5,
    random_state=42,
    n_jobs=-1,
)

print("[train] LightGBM ...")
model = lgb.LGBMRegressor(**LGB_PARAMS)
model.fit(
    X_tr, y_tr,
    eval_set=[(X_va, y_va)],
    eval_metric="rmse",
    callbacks=[
        lgb.early_stopping(stopping_rounds=150),
        lgb.log_evaluation(period=300)
    ]
)

# Use best_iteration if available (safer than default)
best_iter = getattr(model, "best_iteration_", None)
pred_log = model.predict(X_va, num_iteration=best_iter)
pred = np.expm1(pred_log)
y_true = np.expm1(y_va)

# --- Metrics on original scale ---
mae  = mean_absolute_error(y_true, pred)
rmse = mean_squared_error(y_true, pred, squared=False)
mape = (np.abs(y_true - pred) / np.clip(y_true, 1e-9, None)).mean() * 100
print(f"[valid] MAE=${mae:,.0f}  RMSE=${rmse:,.0f}  MAPE={mape:.2f}%  |  best_iter={best_iter}")

# --- Extra sanity: guard against collapsed predictions ---
pred_std = float(np.std(pred))
pred_mean = float(np.mean(pred))
print(f"[sanity] pred mean=${pred_mean:,.0f} | pred std=${pred_std:,.0f}")
if pred_std < 1e-3:
    print("⚠️  WARNING: predictions are (near) constant. Check TE mapping and features.")

# --- Peek a few rows ---
preview = (
    pd.DataFrame({
        "y_true": y_true[:10],
        "y_pred": pred[:10],
        "abs_err": np.abs(y_true[:10] - pred[:10]),
        "pct_err_%": np.abs(y_true[:10] - pred[:10]) / np.clip(y_true[:10], 1e-9, None) * 100
    }).round({"y_true":0, "y_pred":0, "abs_err":0, "pct_err_%":2})
)
print("\n[preview] first 10 validation rows:\n", preview.to_string(index=False))

# --- (Optional) Feature importance: top 15 ---
try:
    fi = pd.Series(model.booster_.feature_importance(importance_type="gain"),
                   index=model.booster_.feature_name())
    top = fi.sort_values(ascending=False).head(15).round(0).astype(int)
    print("\n[top features by gain]\n", top.to_string())
except Exception:
    pass


[sanity] y_tr mean (log): 9.9280  |  expm1 -> $23,256
[sanity] y_va mean (log): 9.9277  |  expm1 -> $23,250
[sanity] model_trim_encoded std: 0.431332  |  NaN% in train: 0.00%
[train] LightGBM ...
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002635 seconds.
Yo

In [23]:
# Cell 7 — Save artifacts (no leakage)
from pathlib import Path
import joblib

ARTIFACT = Path("lgbm_cargurus_pricer_no_leak.joblib").resolve()

num_cols = ["year", "mileage", "engine_displacement", "car_age", "miles_per_year", "log_mileage"]
numeric_medians = df[num_cols].median().to_dict()

artifacts = {
    "model": model,
    "encoding_map": enc_map.to_dict(),
    "global_target_mean": float(y_tr.mean()),
    "model_columns": X_tr.columns.tolist(),
    "num_cols": num_cols,
    "bin_cols": [c for c in ["has_accidents","salvage","theft_title"] if c in X_tr.columns],
    "target_encode_cols": ["model_trim"],
    "numeric_medians": numeric_medians,
}
joblib.dump(artifacts, ARTIFACT)
print(f"[saved] {ARTIFACT}")


[saved] C:\Users\junok\car-price-predictor\notebooks\lgbm_cargurus_pricer_no_leak.joblib


In [24]:
# Cell 8 — Inference + deal rating preview
def preprocess_for_inference(df_raw: pd.DataFrame, artifacts: dict) -> pd.DataFrame:
    enc_map = artifacts["encoding_map"]
    global_mean = artifacts["global_target_mean"]
    model_cols = artifacts["model_columns"]
    num_cols = artifacts["num_cols"]
    bin_cols = artifacts["bin_cols"]
    med = artifacts["numeric_medians"]

    X_new = df_raw.copy()

    # numerics
    for c in ["year","mileage","engine_displacement"]:
        X_new[c] = pd.to_numeric(X_new.get(c, np.nan), errors="coerce").fillna(med[c])

    # derived
    CURRENT_YEAR = 2025
    X_new["car_age"] = (CURRENT_YEAR - pd.to_numeric(X_new["year"], errors="coerce")).clip(lower=0)
    X_new["miles_per_year"] = pd.to_numeric(X_new["mileage"], errors="coerce") / X_new["car_age"].replace(0, np.nan)
    X_new["miles_per_year"] = X_new["miles_per_year"].fillna(0)
    X_new["log_mileage"] = np.log1p(pd.to_numeric(X_new["mileage"], errors="coerce"))

    # binaries
    for b in ["has_accidents","salvage","theft_title"]:
        if b in bin_cols:
            X_new[b] = pd.to_numeric(X_new.get(b, 0), errors="coerce").fillna(0).astype(int)

    # strings for TE
    X_new["model_name"] = X_new.get("model_name", "Unknown").fillna("Unknown").astype(str)
    X_new["trim_name"]  = X_new.get("trim_name",  "Unknown").fillna("Unknown").astype(str)
    X_new["model_trim"] = X_new["model_name"] + "_" + X_new["trim_name"]
    X_new["model_trim_encoded"] = X_new["model_trim"].map(enc_map).fillna(global_mean)

    # drop raw strings; align
    X_new = X_new.drop(columns=["model_name","trim_name","model_trim"], errors="ignore")
    for col in model_cols:
        if col not in X_new.columns:
            X_new[col] = 0
    return X_new[model_cols]

THRESH_EXCELLENT, THRESH_GOOD, THRESH_FAIR = -0.15, -0.05, 0.05
def classify_deal(actual_price: float, predicted_price: float) -> str:
    if pd.isna(actual_price) or pd.isna(predicted_price) or predicted_price <= 0:
        return "Unknown"
    r = (actual_price - predicted_price) / predicted_price
    if r <= THRESH_EXCELLENT: return "Excellent Deal"
    if r <= THRESH_GOOD:      return "Good Deal"
    if r <= THRESH_FAIR:      return "Fair Deal"
    return "Bad Deal"

# quick preview on 10 random rows
artifacts = joblib.load("lgbm_cargurus_pricer_no_leak.joblib")
model = artifacts["model"]

sample = df.sample(n=min(10, len(df)), random_state=2025).copy()
X_inf = preprocess_for_inference(sample, artifacts)
sample["pred_price"] = np.expm1(model.predict(X_inf))
if "price" in sample.columns:
    sample["deal_rating"] = sample.apply(lambda r: classify_deal(r["price"], r["pred_price"]), axis=1)

cols = ["year","mileage","engine_displacement","model_name","trim_name","car_age","miles_per_year","log_mileage"]
for b in ["has_accidents","salvage","theft_title"]:
    if b in df.columns: cols.append(b)
if "price" in sample.columns: cols.append("price")
cols += ["pred_price"]
if "deal_rating" in sample.columns: cols += ["deal_rating"]

print(sample[cols].head(10).to_string(index=False))


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
 year  mileage  engine_displacement     model_name             trim_name  car_age  miles_per_year  log_mileage  has_accidents  salvage  theft_title   price   pred_price deal_rating
 2018  35681.0               1500.0           CR-V                EX FWD        7     5097.285714    10.482402              0        0            0 25600.0 22964.366747    Bad Deal
 2020   8267.0               1400.0           Trax                LT FWD        5     1653.400000     9.020148              0        0            0 24195.0 21355.333206    Bad Deal
 2016  23695.0               1800.0           HR-V                    LX        9     2632.777778    10.0730

In [25]:
import pandas as pd

# 현재 학습에 쓰는 컬럼만 선택
feature_cols = X_tr.columns
corr_matrix = X_tr[feature_cols].corr()

# 절대값 기준 내림차순 정렬
high_corr_pairs = (
    corr_matrix
    .abs()
    .unstack()
    .sort_values(ascending=False)
    .drop_duplicates()
)

# 0.9 이상인 피처쌍만 표시
print("[High correlation pairs > 0.9]:")
print(high_corr_pairs[ (high_corr_pairs < 1.0) & (high_corr_pairs > 0.9) ])


[High correlation pairs > 0.9]:
year  car_age    1.0
dtype: float64
